# Blazing Text multi label Classification example

## Introduction

Multi label text classification is an important aspect of Natural Language Processing, supporting many varied use cases, like search, recommendations, ranking and sentiment analysis. The goal here is to automatically classify new text input into one or more pre-defined categories, for the consuming application to decide how to handle best. Toxicity has been a problem amongst the game community for some time now, and this notebook sets out to solve this problem using SageMakers BlazingText algorithm. 

BlazingText implements the fastText text classification model, optimizing it to leverage GPU acceleration, meaning you can train a model on more than a billion words in a couple of minutes using a multi-core CPU or a GPU, whilst achieving performance on par with the state-of-the-art deep learning text classification algorithms. 

BlazingText on Amazon SageMaker further extends this model with features like Early Stopping and Model Tuning, so that you don’t have to worry about setting the right hyperparameters out of the box.

The following figure, from the [launch announcement blog post](https://aws.amazon.com/blogs/machine-learning/enhanced-text-classification-and-word-vectors-using-amazon-sagemaker-blazingtext/), depicts the simple yet powerful architecture behind the text classification model:

![Architecture](https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2018/07/11/SageMaker-blazingtext-1.gif)

Word embedding is an approach for representing words using a dense vector representation, giving us an improvement over the traditional pure Bag of Words model encoding schemes and their large sparse vectors by using a continuous vector space. The position of a word within the vector space is learned from text and is based on the words that surround the word when it is used, set via the `word_ngrams` HyperParameter. 
The position of a word in this learned vector space is referred to as its embedding, and the number of dimensions it uses are controlled by the `vector_dim` Hyper Parameter. A lookup is done on this embedding layer to get the vector representations of the words in the sentence. The word representations are then averaged into a text representation, which is finally forwarded into a linear classifier. It is virtually impossible to have all the words that we could come across during inference in our training dataset, so generating semantic representations for these words is much more useful than other strategies, like ignoring these words altogether or using random vectors for them. BlazingText can generate meaningful vectors for out-of-vocabulary words by representing their vectors as the sum of the character n-gram vectors, making it, to a degree, tolerant of slang, misspellings, and the introduction of new words to convey familiar meanings.

#### Run this notebook with the Python 3 DataScience kernel on an ml.t3.medium or above instance

The notebook is divided into the following steps

- [Pre-requisite install and setup](#1.\)-Pre-requisite-install-and-setup)
- [Data download and exploration](#2.\)-Data-download-and-exploration)
- [Pre-Processing](#3.\)-Pre-Processing)
- [Training](#4.\)-Training)
- [Deploy](#5.\)-Deploy)
- [Test](#6.\)-Test)
- [Cleanup (optional)](#7.\)-Cleanup)

## 1.) Pre-requisite install and setup

Here we install essential libraries and tools and create some utility functions and helpers

We start by making sure our main dependencies are installed.

In [ ]:
%%bash

deps=("kaggle nltk" "pandas" "sagemaker")
for dep in ${deps[@]}; do
    echo "testing for ${dep}..."
    pip show ${dep} > /dev/null
    if [ $? -ne 0 ]; then
        echo "installing ${dep}" && pip install ${dep}
    else
        echo "found : ${dep}"
    fi
done


Import the general purpose libraries and helpers we need

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.serializers import JSONSerializer
from sagemaker.sklearn import SKLearnModel
from sagemaker.serverless import ServerlessInferenceConfig
from pprint import pprint
import os, sys, random, time, json
import pandas as pd
import nltk, re

## 2.) Data download and exploration

In this segment, we download the dataset locally using the kaggle cli utility and explore it, so we know what we need to do to prepare it for the BlazingText algorithm. 


We need to check for the existence of a kaggle API key, so we can download the dataset.

If this cell fails, please

- sign up for a free kaggle account at www.kaggle.com
- create and download an API key, as per https://www.kaggle.com/docs/api#authentication
- upload it to this directory as 'kaggle.json'
- accept the terms of the competition at [the Kaggle website](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/rules)
- run this cell again

In [ ]:
cwd = os.getcwd()
homeDir = os.path.expanduser("~")
kaggleDir="{}/.kaggle".format(homeDir)
kaggleFile="kaggle.json"
kaggleFQP="{}/{}".format(kaggleDir, kaggleFile)

if not os.path.isdir(kaggleDir):
    print("creating kaggle directory : {}".format(kaggleDir))
    os.mkdir( kaggleDir, 0o755 )

print("checking for kaggle api key : {}".format(kaggleFQP))
if not os.path.isfile(kaggleFQP):
    if not os.path.isfile(kaggleFile):
        print("kaggle api credentials are missing")
        print("you need to create a free API key at kaggle.com and upload it to this directory as '{}'".format(kaggleFile))
        print("please see https://www.kaggle.com/docs/api#authentication for details")
        sys.exit(1)
    else:
        print("found local '{}' file, moving it to : '{}'".format(kaggleFile, kaggleFQP))
        os.rename(kaggleFile, kaggleFQP)
        os.chmod(kaggleFQP, 0o600)
else:
    print("found kaggle api key")

Now we can download the dataset locally using the configured kaggle cli, passing in the competition name as an identifier.

If the cell below fails, make sure you accepted the the terms of the competition at [the Kaggle website](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/rules), which enables your API key for this dataset.

In [ ]:
%%bash

kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

With the dataset downloaded, the next step is to unpack it

In [ ]:
%%bash

ls -l
if [ -f jigsaw-toxic-comment-classification-challenge.zip ]; then

    if [ -d ./data ]; then
        rm -rf ./data
    fi
    echo "unzipping the kaggle dataset"
    unzip ./jigsaw-toxic-comment-classification-challenge.zip -d ./data
    rm -f jigsaw-toxic-comment-classification-challenge.zip
    pushd ./data
    for f in *.zip; do
        echo "unpacking data set : ${f}"
        unzip ${f} && rm -f ${f}
    done
    popd && ls -l ./data
fi

The datasets are not so big, and so we will just load them directly into Pandas DataFrames for further investigation. 

In [ ]:
print("loading: training data")
trainData = pd.read_csv('data/train.csv')
print("loading: test labels")
labelData = pd.read_csv('data/test_labels.csv')
print("loading: full data")
testData = pd.read_csv('data/test.csv')

A brief look tells us that we've actually got 5,066,773 rows of training data with 8 columns, and 2,460,933 rows of test and label data.

In [ ]:
print("(rows, columns) = {}".format(trainData.shape))
print("(rows, columns) = {}".format(labelData.shape))
print("(rows, columns) = {}".format(testData.shape))

Exploring the data sets, we can see in 'train.csv' it has been One Hot Encoded already, with each category of toxicity having its own column, and the test and test_labels datasets are logically joined on the id column. 

There is no column to explicitly represent *'Not toxic'*, and so we will have to handle this in the pre-processor.

The text is raw text, with mis-spellings, slang , plurals, etc, and will need some work doing to it.

In [ ]:
trainData.head(2)

Looking at the test data we see an id and comment text, unprocessed as before.

In [ ]:
testData.head(2)

And so we know the label data must join to the test data set via the id column, hence the matching row count.
We can also see that we need to zero out negative values in the data set.

In [ ]:
labelData.head(2)

## 3.) Pre-Processing

In NLP projects, pre-processing can take up a lot of cpu time and memory, and so we are going to offload it onto bigger resources, using the SageMaker processing feature.

As an overview, we create a script that wraps up what we want to do, and create a Processing job, defining the resources, the data location and the processing script we want run.

Let's start by defining the data locations, where our inout data will be copied to, and our processed data can be found at.

In [ ]:
inputPrefix = "/opt/ml/processing/input"
outputPrefix = "/opt/ml/processing/output"

When you create a SageMaker Processing job, you have to supply a script to take the supplied inputs, transform them as required, and place them into the defined output locations. 

Let's start building the scripts that will do this work for us. The first step is to create a folder to keep everything.

In [ ]:
!mkdir -p scripts

We're going to break out our actual line processing function into a seperate file, to re-use when we deploy our endpoint.

We import all of our NLP pre-processing utilities, the Natural Language Toolkit (nltk), regular expression libraries etc.
We also supress Warnings in Beautiful Soup around url's, as the input is so dirty it can contain anything.

Using the `%%writefile` cell magic, we start by importing some basic libraries, and creating a couple helper functions to also install NLTK, BeautifulSoup etc onto our scikit-learn container, and the compilers they need to build. 
You can append to a file uising the `-a` flag.

In [ ]:
%%writefile scripts/process.py

import nltk
import re
import warnings
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
import contractions
import inflect

warnings.filterwarnings("ignore", category = UserWarning, module = 'bs4')

From nltk we download a tokenizer ruleset called 'punkt', a stopwords collection, our lemmatization rules, and store them locally. 
We then create a stop words collection and a Lemmatizer instance from the downloaded artifacts. 

In [ ]:
%%writefile -a scripts/process.py

print("downloading nltk resources")
downloader = nltk.downloader.Downloader()
downloader._update_index()
downloader.download('punkt')
downloader.download('stopwords')
downloader.download('wordnet')
downloader.download('omw-1.4')

stopWords = stopwords.words('english')
lemma = WordNetLemmatizer()

This is the function that actually does all of our pre-processing on the raw text, using the libraries we prepared earlier.
For each piece of training data, we
- Lower case it
- Strip white space
- Remove markup
- Tokenize it
- For each token we
  - Convert numbers to text
  - Throw away
    - anything shorter than 2 characters
    - anything in the stop words collection
  - Lemmatize it to remove plurals, prefixes, suffixes etc
- Combine the tokens back into a string
- return the result

In [ ]:
%%writefile -a scripts/process.py

def processLine(line):
    
    if isinstance(line, str) == False:
        return ""

    filteredSentence = []
    # lower case everything, and remove basic whitespace
    line = line.lower()
    line = line.strip()
    line = re.sub('\s+', ' ', line)
    # strip html
    beauty = BeautifulSoup(line,'html.parser')
    line = beauty.get_text()
    # strip url's
    line = re.sub(r'https\S', '', line)
    # remove brackets
    line = re.sub('\(.*?\)', '', line)
    # expand contractions
    line = contractions.fix(line)

    for word in word_tokenize(line):
        if (word.isnumeric()):
            try:
                filteredSentence.append(inflect.engine().number_to_words(word))
            except Exception as e:
                # print("Couldn't convert number : '{}'".format(word))
                pass

        elif (len(word) > 2) and (word not in stopWords):
            fixed = lemma.lemmatize(word, pos = wordnet.VERB)
            filteredSentence.append(fixed)
    result = " ".join(filteredSentence)
    return result

And that is our core processor written out. 

Now we can start on the wrapper script we will pass to our Processing job.
Initially, we create a utility function to get our helper libraries installed onto our Processing environment.

In [ ]:
%%writefile scripts/preprocessing.py
import subprocess
import sys
import os
import argparse
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def pipInstall(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("installing : NLTK")
pipInstall ("nltk")
print("installing : BeautifulSoup")
pipInstall("bs4")
print("installing : Inflect")
pipInstall("inflect")
print("installing : Contractions")
pipInstall("contractions")

At this point, we want to inline our line processing function, since we can only pass a single file to our Processing environment.

In [ ]:
!cat scripts/process.py >> scripts/preprocessing.py

This is a simple utility function to convert a training row into the format that BlazingText expects, 
prepending the text with the category labels, and then adding the processed text.

We also create a dictionary to hold our Classification Labels lookup.

In [ ]:
%%writefile -a scripts/preprocessing.py

indexToLabel = {}

def transformInstance(row):
    resultRow = []
    for i in range(2, 8):
        if ( row.iloc[i] > 0 ):
            label = "__label__" + indexToLabel[str(i - 1)]
            resultRow.append(label)
    
    # add a 'none' label if we didn't get any specific category assigned
    if len(resultRow) == 0:
        resultRow.append("__label__none")

    text = processLine(row[1])
    resultRow.append(text)
    return ' '.join(map(str, resultRow))

At this point, we need a mapper between a one hot encoded column and a category, based on column index, 
so we create a dictionary to represent this mapping.

In [ ]:
%%writefile -a scripts/preprocessing.py

def createLabels(labels):
    print("generating labels")
    for i in range(1, 7):
        label = labels.iloc[:, i].name
        indexToLabel[str(i)] = label.strip()

Another simple utility function to set the None category on the label data set. A `negative one` value in the label data means that row wasn't used for measuring, and so all we care about is if all our columns are zero value then we are not toxic.

In [ ]:
%%writefile -a scripts/preprocessing.py

def calcNone(row):
    for v in row.loc["toxic":"identity_hate"]:
        if (v < 0): return -1
        if (v > 0): return 0
    return 1

Now we get to the entry point for the Pre-Processing script.
This takes the following steps...

- checks for any args that can override the default data input and output locations
- checks that the files have landed from S3
- creates our Classification Label stems
- pushes each line of training data thru our filter
- shuffles the result
- splits into 90% training and 10% validation data sets
- writes these datasets to csv, ready for uploading to S3 by SageMaker

For convenience, we also combine the `test.csv` and `test_labels.csv` datasets into a new one, matching on the `id` column, and drop any rows containing a `negative one` value, as these weren't used for scoring the competition. 
We start by setting a common index, and concatenating them together.

In [ ]:
%%writefile -a scripts/preprocessing.py

if __name__ == "__main__":

    # handle any args
    parser = argparse.ArgumentParser()
    parser.add_argument("--inputPrefix", type = str, default = "/opt/ml/processing/input")
    parser.add_argument("--outputPrefix", type = str, default = "/opt/ml/processing/output")
    args, _ = parser.parse_known_args()

    print("Received arguments {}".format(args))

    # capture our paths
    inputPrefix = args.inputPrefix
    outputPrefix = args.outputPrefix

    print("checking files are local, and getting their sizes...")
    if not os.path.isdir(inputPrefix):
        print("input prefix directory doesn't exist : {}".format(inputPrefix))

    fileList = filter( lambda x: os.path.isfile(os.path.join(inputPrefix, x)), os.listdir(inputPrefix) )
    fileSizes = [ 
        (fileName, os.stat(os.path.join(inputPrefix, fileName)).st_size) 
        for fileName in fileList  
    ]
    for fileName, size in fileSizes:
        print(inputPrefix, ' --> ', fileName, ' --> ', size, ' bytes')     

    # Now we load all of the datasets into pandas data frames, to work with
    trainPath = os.path.join(inputPrefix, "train.csv")
    print("loading: training data from : {}".format(trainPath))
    trainData = pd.read_csv(trainPath)
    testPath = os.path.join(inputPrefix, "test.csv")
    print("loading: test data from : {}".format(testPath))
    testData = pd.read_csv(testPath)
    labelsPath = os.path.join(inputPrefix, "test_labels.csv")
    print("loading: test labels data from : {}".format(labelsPath))
    labelData = pd.read_csv(labelsPath)

    # popuate the labels lookup
    createLabels(labelData)

    # kick off the pre-processing
    # with the training data, 
    # writing the result into a new column.
    print("pre-processing {} rows of training data".format(trainData.shape[0]))
    trainData["processed"] = trainData.apply(lambda row: transformInstance(row), axis = 1)

    # Next we want to pre-process the test data as well, also writing it into a new column
    print("pre-processing {} rows of test data".format(testData.shape[0]))
    testData["processed"] = testData.apply(lambda row: processLine(row[1]), axis = 1)
    
    print("pre-processing {} rows of test_labels data".format(labelData.shape[0]))
    # calculate the none field
    labelData["none"] = labelData.apply(lambda row: calcNone(row), axis = 1)
    # join the 2 datasets into one linked one
    if testData.index.name != "id":
        testData.set_index("id", inplace = True)
    if labelData.index.name != "id":
        labelData.set_index("id", inplace = True)
    mergedDf = pd.concat([testData, labelData], axis = 1)
    
    # and drop anything that wasn't used in the competition marking
    mergedDf = mergedDf.loc[mergedDf["toxic"] != -1]
    print(mergedDf.head(10))
    print("done pre-processing")

    # Shuffle the data in case it is grouped
    shuffle(trainData)

    # Split into train and validation data sets
    train, validation = train_test_split(trainData, test_size = 0.1)
    print("train:validation split = {}:{} rows".format(train.shape[0], validation.shape[0]))

    try:
        print("creating output directory : {}".format(outputPrefix))
        if not os.path.isdir(outputPrefix):
            os.makedirs(outputPrefix)
        print("successfully created directories")
    except Exception as e:
        print(e)
        print("could not make directories")
        pass

    train.to_csv(os.path.join(outputPrefix, "train.csv"), columns = ["processed"], header = False, index = False)
    validation.to_csv(os.path.join(outputPrefix, "validation.csv"), columns=["processed"], header = False, index = False)
    trainData.to_csv(os.path.join(outputPrefix, "full.csv"), index = False)
    mergedDf.to_csv(os.path.join(outputPrefix, "test.csv"), index = True)

    print("csv files successfully written to : {}".format(outputPrefix))

Now we have the pre-processing script, we can create the job definition, and run it on a suitably sized machine. 
This brings our processing time down to around 36 minutes from over 100 minutes if run locally on an ml.t3.medium, allowing us to keep our Jupyter notebook kernel as small and economic as possible. 

In [ ]:
%%time
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
role = get_execution_role()

sklearnProcessor = SKLearnProcessor(
    framework_version = "0.23-1", role = role, instance_type = "ml.c5.4xlarge", instance_count = 1
)

sklearnProcessor.run(
    code = "scripts/preprocessing.py",
    inputs = [
        ProcessingInput(source = "data", destination = inputPrefix, input_name = "input")
    ], 
    outputs = [
        ProcessingOutput(source = outputPrefix, output_name = "output")
    ], 
    arguments = ["--inputPrefix", inputPrefix, "--outputPrefix", outputPrefix]
)

To load the pre-processed data for validation, and for our subsequent training job, we need the S3 path the results were stored to. We can get from the Processing Job Description, as follows.

In [ ]:
jobDescription = sklearnProcessor.jobs[-1].describe()
outputConfig = jobDescription["ProcessingOutputConfig"]
print(json.dumps(outputConfig, indent = 2))
output = outputConfig["Outputs"][0]
s3Path = output["S3Output"]["S3Uri"]

At this point we can load a sample from our pre-processed datasets into pandas data frames directly from S3, to check the pre-processing is as expected.

In [ ]:
%%time
s3TrainData = os.path.join(s3Path, 'train.csv')
s3ValidationData = os.path.join(s3Path, 'validation.csv')
print("loading: full data")
s3FullData = os.path.join(s3Path, 'full.csv')
fullData = pd.read_csv(s3FullData)
print("loading: test data")
s3TestData = os.path.join(s3Path, 'test.csv')
testData = pd.read_csv(s3TestData)

Now we can compare the before and after states from pre-processing the text, and confirm the results [look good](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html) for BlazingText to train on.

In [ ]:
test, processed = fullData.iloc[2]["comment_text"], fullData.iloc[2]["processed"]
print("raw : '{}'\npre-processed : '{}'".format(test, processed))

## 4.) Training

Time to start setting up SageMaker for training on our prepared data sets. 

To do this, we define a prefix for that bucket, to isolate this workload
and then we grab 

- a reference to our session
- our default bucket name
- the role that SageMaker is running under

In [ ]:
prefix = 'toxicity/blazingtext'
sess = sagemaker.Session()
bucket = sess.default_bucket()
s3OutputLocation = 's3://{}/{}/output'.format(bucket, prefix)

We can tell SageMaker where to find its training and validation data via the `data_channels` construct. 

This requires 
- the path to the data in S3
- the distribution mode of the data. `FullyReplicated` means that each training node will get a full copy of the data. The alternate setting is `ShardedByS3Key`, for when you are training with a cluster and can optimize by partitioning the data. We can only use one trainig node with the Classification mode of Blazing Text, and so we select `FullyReplicated`.
- the content (mime) type of the data
- The S3 data type, being one of `S3Prefix`, `ManifestFile`, `AugmentedManifestFile`. This denotes what the first parameter is pointing to, in our case, an S3 Bucket and key prefix.


In [ ]:
trainInput = sagemaker.inputs.TrainingInput(
    s3TrainData, 
    distribution = 'FullyReplicated', 
    content_type = 'text/plain', 
    s3_data_type = 'S3Prefix')

validationInput = sagemaker.inputs.TrainingInput(
    s3ValidationData, 
    distribution = 'FullyReplicated', 
    content_type = 'text/plain', 
    s3_data_type = 'S3Prefix')

dataChannels = { 'train': trainInput, 'validation': validationInput }

The last piece of information we need to configure our training job is the Fully Qualified `URI` for the latest public BlazingText container from the ECR service, in our current region.

In [ ]:
regionName = sess.boto_region_name
container = sagemaker.image_uris.retrieve("blazingtext", regionName, "latest", py_version = "py3")
print('Using SageMaker BlazingText container: {} ({})'.format(container, regionName))

With this information, we can define the SageMaker training job itself, by setting our [algorithm specific hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) and the generic SageMaker training parameters via the standard [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) construct.

The Algorithm Specic Hyperparameters for Blazing Text in Classification mode can be sumarized as


| Parameter Name | Parameter Type              | Recommended Ranges or Values    | Required | Default | Description                                                                                                                                       |   |
|----------------|-----------------------------|---------------------------------|----------|---------|---------------------------------------------------------------------------------------------------------------------------------------------------|---|
| buckets        | `IntegerParameterRange`     | [1000000-10000000]              | N        | 2000000 | The number of hash buckets to use for word n-grams                                                                                                |   |
| early_stopping | `Boolean`                   | [`True`, `False`]               | N        | `False`   | Whether to stop training if validation accuracy doesn't improve after a patience number of epochs                                                 |   |
| epochs         | `IntegerParameterRange`     | [5-15]                          | N        | 5       | The maximum number of complete passes through the training data                                                                                   |   |
| learning_rate  | `ContinuousParameterRange`  | MinValue: 0.005, MaxValue: 0.01 | N        | 0.05    | The step size used for parameter updates                                                                                                          |   |
| min_count      | `IntegerParameterRange`     | [0-100]                         | N        | 5       | Words that appear less than min_count times are discarded                                                                                         |   |
| min_epochs     | `IntegerParameterRange`     | [0-100]                         | N        | 5       | The minimum number of epochs to train before early stopping logic is invoked                                                                      |   |
| mode           | `CategoricalParameterRange` | [`supervised`]                | Y        | N/A     | The training mode                                                                                                                                 |   |
| patience       | `IntegerParameterRange`     | [0-100]                         | N        | 4       | The number of epochs to wait before applying early stopping when no progress is made on the validation set. Used only when early_stopping is True |   |
| vector_dim     | `IntegerParameterRange`     | [32-300]                        | N        | 100     | The dimension of the embedding layer                                                                                                              |   |
| word_ngrams    | `IntegerParameterRange`     | [1-3]                           | N        | 2       | The number of word n-gram features to use                                                                                                         |   |

You can see from the size of our hyper parameters construct that we are accepting most of the defaults, we set the training mode to supervized and then only override the values for 

- number of epochs the training should run for
- minimum word count
- the number of vector dimensions

For the general Estimator properties, we are using one instance of an ml.c5.4xlarge, with a default 30GB EBS Volume attached, and sending the data via `File` mode, where the process starts off by pulling all the data from S3 to the local attached EBS volume and ingests it as a file from there.

In [ ]:
hp = {
    "mode": "supervised",
    "epochs": 50,
    "min_count": 2,
    "vector_dim": 10
}

bt_estimator = sagemaker.estimator.Estimator(container,
                                         role, 
                                         instance_count = 1, 
                                         instance_type = 'ml.c5.4xlarge',
                                         input_mode = 'File',
                                         output_path = s3OutputLocation,
                                         hyperparameters = hp)

We can now invoke the training, passing in our training and validation data set path, from S3.



The supervized BlazingText algorithm measures success by looking to maximize accuracy, defined as the classification accuracy on the user-specified validation dataset

You should achieve something in the area of 94% accuracy using the hyperparameter settings from above, after around just 5 minutes of elapsed training time.

In [ ]:
bt_estimator.fit(inputs = dataChannels, logs = True)

With training completed, we can retrieve the location in s3 where our model artifact has been saved.

In [ ]:
training_job_name = bt_estimator.latest_training_job.name
training_job_info = sess.sagemaker_client.describe_training_job(TrainingJobName = training_job_name)
s3_model_artifact = training_job_info['ModelArtifacts']['S3ModelArtifacts']

## 5.) Deploy

We can now go about deploying the model into production, using initially one instance of an ml.t2.large virtual machine. For real traffic volumes you would likely use a bigger machine.

Because we are optimizing for cost, and we want the consumer of our endpoint to be freed from knowing and re-implementing our pre-processing rules, we will take our BlazingText trained model, and host it in a generic SciKit Learn container, using fastText and our pre-processing code from before. 
The provided algorithm containers do not support injecting your own code, but the framework containers do, and provide a number of lifecycle hooks for us to implement what we need.

First, we create a requirements.txt file that holds our dependencies, that the SageMaker platform will install for us at launch.

For more on this, please see [Using Scikit-learn with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html)

In [ ]:
%%writefile scripts/requirements.txt
fastText
numpy
sklearn
pandas
nltk
bs4
inflect
contractions
sagemaker

The last component is to build our pre-processing code up, as before.

We start with some standard imports, and pull in our processLine function that we created before.

In [ ]:
%%writefile scripts/serve.py
import json
import os
import fasttext as ft
import numpy as np
from process import processLine
from sagemaker.serializers import CSVSerializer
import multiprocessing
import os

cpu_count = multiprocessing.cpu_count()
model_server_timeout = os.environ.get('MODEL_SERVER_TIMEOUT', 120)
model_server_workers = int(os.environ.get('MODEL_SERVER_WORKERS', cpu_count))

CONTENT_TYPE_JSON = 'application/json'
CONTENT_TYPE_CSV = "text/csv"

At this point we just need to implement the SageMaker lifecycle hooks to achieve 4 objectives. 
The first is to load our model.

In [ ]:
%%writefile -a scripts/serve.py

def model_fn(model_dir):
    model_path = os.path.join(model_dir, "model.bin")
    if not os.path.isfile(model_path):
        print("model_fn : no model at : {}".format(model_path))
        for everything in os.listdir(model_dir):
            print("model_fn : found file : {}".format(everything))
        return None
    return ft.load_model(model_path)

The second is to transform our input from JSON

In [ ]:
%%writefile -a scripts/serve.py

def input_fn(serialized_input_data, content_type = CONTENT_TYPE_JSON):
    #print("input_fn : {}".format(serialized_input_data))
    #print("input_fn : Using '{}' Parser".format(content_type))
    if (CONTENT_TYPE_JSON == content_type):
        input_data = json.loads(serialized_input_data)
        return input_data
    elif (CONTENT_TYPE_CSV == content_type):
        input_data = {
            "configuration": { "k": 7 }, 
            "instances" : serialized_input_data.splitlines()
        }
        return input_data
    else:
        raise Exception('Unsupported content type')

Third, we need to implement the prediction call. Here we are looping thru all of the submitted lines for inference, and passing them thru our pre-processor function, before calling into the model.

In [ ]:
%%writefile -a scripts/serve.py

def predict_fn(input_data, model):
    k = 2
    processed = []
    #print("predict_fn : ".format(input_data))
    if not input_data["configuration"] is None:
        config = input_data["configuration"]
        if not config["k"] is None:
            k = int(config["k"])
    if not input_data["instances"] is None:
        instances = input_data["instances"]
        for line in instances:
            processedLine = processLine(line)
            processed.append(processedLine)

    prediction = model.predict(processed, k)
    return np.array(prediction)

Last, we transform the results back into the correct output format

In [ ]:
%%writefile -a scripts/serve.py

def output_fn(prediction_output, accept = CONTENT_TYPE_JSON):
    #print("output_fn : Using '{}' Parser".format(accept))
    if CONTENT_TYPE_JSON ==  accept:
        return json.dumps(prediction_output.tolist()), accept
    elif CONTENT_TYPE_CSV == accept:
        return CSVSerializer().serialize(prediction_output.tolist()), accept
    raise Exception('Unsupported content type')

This gives us everything we need to create a Model object to deploy. 
We copy the whole `scripts` folder across, and set the entry point to be our `serve.py` file.

In [ ]:
sklearn_model = SKLearnModel(
    model_data = s3_model_artifact,
    source_dir = "scripts", 
    entry_point = "serve.py",
    framework_version = "0.23-1",
    role = role
)

We'll use the new [Serverless Inference feature](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html), so we can leave our endpoint deployed and only be charged when we actually make predictions

In [ ]:
serverless_config = ServerlessInferenceConfig(
  memory_size_in_mb = 2048,
  max_concurrency = 5,
)

We attach a JSONSerializer for marshalling JSON inputs to the inference endpoint in its native format, and deploy the endpoint. 

(Because we are optimizing for cost and using the scikit learn container as a base, we need to install all of our dependencies at deployment time, which can add some time overhead to our deployment. In a production environment, you would either want to create a finalised container for using with serverless inference, or use the auto-scaling server backed option.)

In [ ]:
classifier_endpoint = sklearn_model.deploy(
    serverless_inference_config = serverless_config,  
    serializer = JSONSerializer()
)

Here we get back the SageMaker Endpoint name, which we use for testing, and you would also use if you were to invoke the endpoint from a Lambda function or similar, using the AWS SDK

In [ ]:
endpoint_name = classifier_endpoint.endpoint_name
endpoint_name

## 6.) Test

At this stage we have a live inference endpoint available to us, so we can run some test queries against it. 

Let's specify the index so we can pull specific examples out by their id.

In [ ]:
if testData.index.name != "id":
    testData.set_index("id", inplace = True)

First up, we'll test a potentially contentious sentence, that has some potential trigger words it, but no malice.

We want to test with raw examples, and make sure our pre-processing happens on the inference side, so we can just create a Plain Old Javascript Object (POJO) and send it off to our inference endpoint by calling `predict`, and finally print the results back.

In [ ]:
simpleTest = testData.loc["001d39c71fce6f78", "comment_text":"none"]
print(simpleTest)

We pack this example inside a POJO as the `instances` property. 

With this payload we invoke the endpoint, and print the results out.

We can see from the results, with the `None` category gettng the highest weighting, that our model wasn't fooled by this simple test.

In [ ]:
runtime_client = boto3.client("runtime.sagemaker")
payload =  { "instances" : [simpleTest.comment_text], "configuration" : { "k": 1 } }
response = runtime_client.invoke_endpoint(
    EndpointName = endpoint_name, ContentType = "application/json", Body = json.dumps(payload)
)
response_payload = json.loads(response['Body'].read().decode("utf-8"))

In [ ]:
print ("response_payload: {}".format(pprint(response_payload)))

Finally, let's try making many predictions against our endpoint using a mix of samples pulled from our test dataset.
We create a helper function here to find us a random example of a toxic or non-toxic sentence, determined by the functions only parameter.

In [ ]:
testRowCount = testData.shape[0]
def findTestCandidate(toxic):
    randId = random.randint(0, testRowCount - 1)
    result = testData.iloc[randId]
    if type(result["comment_text"]) is str:
        if result.none != toxic:
            return result
    return findTestCandidate(toxic)

Now we can test our endpoint, pulling back 4 categories by using the `configuration.k` parameter. 
This option tells the Blazing Text Classification algorithm how many matched categories with weighting we want to get back in each result.

You can run this cell as often as you like, and experiment with changing the `True` to `False` to select non-toxic examples.

In [ ]:
result = findTestCandidate(True)
payload = {
            "instances" : [result.processed],
            "configuration": { "k": 4 }
        }
# uncomment this next line if you are ok seeing the sentence
# print(json.dumps(payload, indent = 2))
response = runtime_client.invoke_endpoint(
    EndpointName = endpoint_name, ContentType = "application/json", Body = json.dumps(payload)
)
response_payload = json.loads(response['Body'].read().decode("utf-8"))

In [ ]:
print ("response_payload: {}".format(response_payload))

## 7.) Cleanup

Finally, we can optionally delete the serverless inference endpoint, but this isn't essential

In [ ]:
#classifier_endpoint.delete_endpoint()

And that's it :)